# Gibbs sampling for naive bayes

Implementation of the example in paper _Gibbs Sampling for the Uninitiated_

In [107]:
# Generative model
import numpy as np
from scipy.stats import dirichlet

real_pi = 0.3
vocabulary = np.array(['sector', 'twijfels', 'taxivervoer', 'uitgangspunt', 'belangen'])

real_theta_zero = np.random.dirichlet(np.ones(len(vocabulary)))
real_theta_one = np.random.dirichlet(np.ones(len(vocabulary)))

print real_theta_zero, sum(theta_zero)
print real_theta_one, sum(theta_one)

[ 0.3534365   0.02238845  0.16278148  0.3446405   0.11675307] 1.0
[ 0.37301648  0.14526872  0.18574284  0.01291135  0.28306062] 1.0


In [108]:
# generate documents
import random

def generate_document(n_words=10):
    label = int(random.random() < pi)
    
    if label == 0:
        d = real_theta_zero
    else:
        d = real_theta_one
    
    words = []
    
    for i in range(n_words):
        word = np.random.multinomial(1, d)
        index = np.where(word==1)[0][0]
        words.append(vocabulary[index])
    
    return (label, words)

corpus = [generate_document() for i in range(10)]

real_labels = [doc[0] for doc in corpus]

documents = [doc[1] for doc in corpus]

for doc in corpus:
    print doc[0], doc[1]


1 ['belangen', 'sector', 'sector', 'belangen', 'sector', 'belangen', 'sector', 'belangen', 'sector', 'sector']
1 ['belangen', 'sector', 'belangen', 'belangen', 'sector', 'sector', 'taxivervoer', 'twijfels', 'belangen', 'taxivervoer']
1 ['taxivervoer', 'sector', 'sector', 'belangen', 'belangen', 'sector', 'sector', 'belangen', 'taxivervoer', 'sector']
1 ['sector', 'belangen', 'sector', 'taxivervoer', 'belangen', 'twijfels', 'belangen', 'belangen', 'belangen', 'twijfels']
1 ['twijfels', 'belangen', 'taxivervoer', 'twijfels', 'twijfels', 'sector', 'twijfels', 'sector', 'sector', 'belangen']
1 ['belangen', 'sector', 'belangen', 'belangen', 'twijfels', 'belangen', 'sector', 'belangen', 'taxivervoer', 'belangen']
0 ['sector', 'taxivervoer', 'taxivervoer', 'uitgangspunt', 'twijfels', 'sector', 'sector', 'sector', 'uitgangspunt', 'uitgangspunt']
1 ['sector', 'belangen', 'twijfels', 'sector', 'taxivervoer', 'sector', 'twijfels', 'belangen', 'sector', 'sector']
1 ['taxivervoer', 'sector', 'secto

In [109]:
import operator
import functools

def get_p(theta, num):
    # equation 49
    return (num + 1.0 - 1)/(len(corpus) + 1 + 1 -1) * functools.reduce(operator.mul, [pow(float(theta[i]), C[corpus[j][1][i]]) for i in range(len(vocabulary))], 1)
    

In [116]:
# gibbs sampling

from collections import Counter
import sys

# initialization
print 'Initialization'
pi = random.random()
print pi

theta_zero = np.random.dirichlet(np.ones(len(vocabulary)))
theta_one = np.random.dirichlet(np.ones(len(vocabulary)))

print theta_zero
print theta_one

L = [int(random.random() < pi) for i in range(len(corpus))]
num_ones = sum(L)
num_zeros = len(L) - num_ones

print L
#print '1:', num_ones
#print '0:', num_zeros

# count words in documents
corpus_0_counts = Counter()
corpus_1_counts = Counter()
document_counts = []
for i in range(len(corpus)):
    if L[i] == 0:
        corpus_0_counts.update(corpus[i][1])
    else:
        corpus_1_counts.update(corpus[i][1])
        
    document_counts.append(Counter(corpus[i][1]))

iterations = 10000
theta_zeros = np.zeros((iterations, len(theta_zero)))
theta_ones = np.zeros((iterations, len(theta_one)))

for t in range(0, iterations):
    for j in range(0, len(corpus)):
        #print 'text', j, 'label', L[j], 
        label_j = L[j]
        if label_j == 0:
            corpus_0_counts.subtract(document_counts[j])
            C = corpus_0_counts
            num_zeros -= 1
        else:
            corpus_1_counts.subtract(document_counts[j])
            C = corpus_1_counts
            num_ones -= 1

        P_label_0 = get_p(theta_zero, num_zeros)
        P_label_1 = get_p(theta_one, num_ones)
        
        #print P_label_0
        #print P_label_1
        
        P = P_label_0/(P_label_0+P_label_1)
        #print P
        if P > random.random():
            L[j] = 0
            num_zeros += 1
            corpus_0_counts.update(document_counts[j])
            
            #print 'New label = 0'
        else:
            L[j] = 1
            num_ones += 1
            corpus_1_counts.update(document_counts[j])
            #print 'New label = 1'
    # resample thetas
    wc_0 = [corpus_0_counts[w] + 1 for w in vocabulary]
    #print wc_0
    theta_zero = np.random.dirichlet(wc_0)
    theta_zeros[t] = theta_zero
    
    wc_1 = [corpus_1_counts[w] + 1 for w in vocabulary]
    #print wc_1
    theta_one = np.random.dirichlet(wc_1)
    theta_ones[t] = theta_one

print 'Found parameters'
print np.mean(theta_zeros, axis=0)
print np.mean(theta_ones, axis=0)

print 'Real parameters'
print real_theta_zero
print real_theta_one

Initialization
0.500304700895
[ 0.02174525  0.0181391   0.73945465  0.01253435  0.20812665]
[ 0.00832763  0.0152612   0.01704993  0.03044369  0.92891755]
[0, 0, 1, 0, 1, 1, 1, 1, 0, 0]
Found parameters
[ 0.38060674  0.14339653  0.1429708   0.03812048  0.29490545]
[ 0.20075403  0.19944209  0.20180688  0.1999485   0.1980485 ]
Real parameters
[ 0.3534365   0.02238845  0.16278148  0.3446405   0.11675307]
[ 0.37301648  0.14526872  0.18574284  0.01291135  0.28306062]
